**Project Documentation: Time Series Analysis**
-------
**Objective:**
Using the information gained from the EDA work, we can now complete the prediction task at hand.   

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima

In [16]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Load the data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [18]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['day'] = pd.to_datetime(train['date']).dt.day

In [19]:
df = train

In [20]:
df = df.drop(['id'], axis = 1)
df = df.dropna()
df['product_country'] = df['product'] + "_" + df['country']
df['product_store'] = df['product'] + "_" + df['store']
df['country_store'] = df['country'] + "_" + df['store']
df.head(5)

,date,country,store,product,num_sold,year,month,day,product_country,product_store,country_store
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0,2010,1,1,Kaggle_Canada,Kaggle_Discount Stickers,Canada_Discount Stickers
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0,2010,1,1,Kaggle Tiers_Canada,Kaggle Tiers_Discount Stickers,Canada_Discount Stickers
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0,2010,1,1,Kerneler_Canada,Kerneler_Discount Stickers,Canada_Discount Stickers
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0,2010,1,1,Kerneler Dark Mode_Canada,Kerneler Dark Mode_Discount Stickers,Canada_Discount Stickers
5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0,2010,1,1,Holographic Goose_Canada,Holographic Goose_Stickers for Less,Canada_Stickers for Less


In [30]:
aggregated_df = df.groupby(['country', 'store', 'product', 'date'])['num_sold'].sum().reset_index()

# Add year, month, and day columns
aggregated_df['year'] = aggregated_df['date'].dt.year-2010
aggregated_df['month'] = aggregated_df['date'].dt.month
aggregated_df['day'] = aggregated_df['date'].dt.day

# Create new combined columns
aggregated_df['product_country'] = aggregated_df['product'] + "_" + aggregated_df['country']
aggregated_df['product_store'] = aggregated_df['product'] + "_" + aggregated_df['store']
aggregated_df['country_store'] = aggregated_df['country'] + "_" + aggregated_df['store']

# Display the first few rows of the transformed data
aggregated_df.head(5)

,country,store,product,date,num_sold,year,month,day,product_country,product_store,country_store
0,Canada,Discount Stickers,Kaggle,2010-01-01,973.0,0,1,1,Kaggle_Canada,Kaggle_Discount Stickers,Canada_Discount Stickers
1,Canada,Discount Stickers,Kaggle,2010-01-02,881.0,0,1,2,Kaggle_Canada,Kaggle_Discount Stickers,Canada_Discount Stickers
2,Canada,Discount Stickers,Kaggle,2010-01-03,1003.0,0,1,3,Kaggle_Canada,Kaggle_Discount Stickers,Canada_Discount Stickers
3,Canada,Discount Stickers,Kaggle,2010-01-04,744.0,0,1,4,Kaggle_Canada,Kaggle_Discount Stickers,Canada_Discount Stickers
4,Canada,Discount Stickers,Kaggle,2010-01-05,707.0,0,1,5,Kaggle_Canada,Kaggle_Discount Stickers,Canada_Discount Stickers


In [25]:
# Step 1: Prepare the data
aggregated_df.set_index('date', inplace=True)  # Set 'date' as the index
aggregated_df.sort_index(inplace=True)  # Ensure data is sorted by date

# Use 'num_sold' for time-series analysis
time_series = aggregated_df['num_sold']

# Step 2: Check for stationarity
from statsmodels.tsa.stattools import adfuller

adf_test = adfuller(time_series.dropna())  # Handle any missing values
print("ADF Statistic:", adf_test[0])
print("p-value:", adf_test[1])
if adf_test[1] > 0.05:
    print("Series is not stationary. Applying differencing.")
    time_series = time_series.diff().dropna()  # Apply differencing if needed

ADF Statistic: -38.363257171917105
p-value: 0.0


In [26]:
# Step 3: Fit ARIMA model
# Automatically determine ARIMA parameters
stepwise_fit = auto_arima(time_series, seasonal=False, trace=True, suppress_warnings=True)
print(stepwise_fit.summary())

# Fit the ARIMA model with the optimal parameters
model = ARIMA(time_series, order=stepwise_fit.order)
model_fit = model.fit()
print(model_fit.summary())

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=131.74 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3674259.382, Time=2.42 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3610236.086, Time=3.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=51.92 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3674257.382, Time=1.22 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3584026.298, Time=4.06 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=3569418.408, Time=5.04 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=3560175.387, Time=13.78 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=3554044.450, Time=16.63 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=219.75 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=190.63 sec
 ARIMA(5,1,0)(0,0,0)[0]             : AIC=3554042.450, Time=8.08 sec
 ARIMA(4,1,0)(0,0,0)[0]             : AIC=3560173.387, Time=6.58 sec
 ARIMA(5,1,1)(0,0,0)[0]             : AIC=inf, Time=147.05 sec
 ARIMA(4,1,1)(0,0,0)

C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               num_sold   No. Observations:               221259
Model:                 ARIMA(5, 1, 0)   Log Likelihood            -1777015.225
Date:                Fri, 10 Jan 2025   AIC                        3554042.450
Time:                        15:07:40   BIC                        3554104.293
Sample:                             0   HQIC                       3554060.572
                             - 221259                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.8379      0.002   -416.878      0.000      -0.842      -0.834
ar.L2         -0.6721      0.003   -264.416      0.000      -0.677      -0.667
ar.L3         -0.5052      0.003   -187.088      0.0

In [27]:
# Step 4: Forecast
forecast_steps = 10  # Define how many steps to forecast
forecast = model_fit.forecast(steps=forecast_steps)
print(f"Forecast for next {forecast_steps} steps:\n", forecast)

Forecast for next 10 steps:
 221259    636.369432
221260    661.570497
221261    657.100850
221262    749.205285
221263    707.463488
221264    786.052261
221265    699.057503
221266    710.073377
221267    718.376463
221268    728.518134
Name: predicted_mean, dtype: float64


C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [29]:
from sklearn.metrics import mean_absolute_percentage_error


# Step 5: Evaluate the model
# Split data into training and test sets (optional for evaluation)
train_size = int(len(time_series) * 0.8)
train, test = time_series[:train_size], time_series[train_size:]
model = ARIMA(train, order=stepwise_fit.order).fit()
forecast_test = model.forecast(steps=len(test))
mape = mean_absolute_percentage_error(test, forecast_test)

print(f"MAPE on test data: {mape:.2%}")

C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\maggi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\U

MAPE on test data: 373.88%
